In [3]:
import geemap
import ee
from GEE_xiong import ee_export
import os
import pandas as pd

# 创建一个空的 DataFrame
df = pd.DataFrame()
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
#研究区
# f_shp =r"D:\work\stld\gx\boundary.shp" #添加面shp（文件名不能有“_”和中文）
# f_shp_ee = geemap.shp_to_ee(f_shp)
# Map.addLayer(f_shp_ee, {}, 'GX')

#添加研究区
shp = r"D:\work\stld\gx\temp\prj.shp"
shp_ee = geemap.shp_to_ee(shp)
Map.addLayer(shp_ee, {}, 'GXs')
# roi = shp_ee.geometry()                       


In [5]:
#去云
def maskL457sr(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Unused
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

  # Replace the original bands with the scaled ones and apply the masks.
  return image.addBands(opticalBands, None, True) \
      .addBands(thermalBand, None, True) \
      .updateMask(qaMask) \
      .updateMask(saturationMask)

In [6]:
#添加ndvi波段
def ndvi(img):
    ndvi = img.normalizedDifference(['SR_B4','SR_B3']).rename('NDVI')
    Msk = ndvi.lte(1).And(ndvi.gt(0))
    ndvi = ndvi.updateMask(Msk)
    return img.addBands(ndvi).copyProperties(img, ["system:time_start",'system:id'])

In [7]:
#添加ndvi最大、最小、均值三个属性
def properties(img):  
    percentile = img.reduceRegions(**{
        'collection': shp_ee,
        'reducer': ee.Reducer.percentile([5, 95]),#百分之五到百分之九十五
        'scale': 30,
        })
    p5 = ee.Number(percentile.first().get("p5"))
    p95 = ee.Number(percentile.first().get("p95"))
    
#     xmin = img.reduceRegion(**{#**以dic形式传递，不加则以元组形式
#       'reducer': ee.Reducer.min(),
#       'geometry': roi,
#       'scale': 30,
#       'maxPixels': 1e13
#     }).get('NDVI')
#     xmax = img.reduceRegion(**{
#           'reducer': ee.Reducer.max(),
#           'geometry': roi,
#           'scale': 30,
#           'maxPixels': 1e13
#         }).get('NDVI')
    mean = img.reduceRegion(**{#**以dic形式传递，不加则以元组形式
      'reducer': ee.Reducer.mean(),
      'geometry': roi,
      'scale': 30,
      'maxPixels': 1e13
    }).get('NDVI')
    return img.set({"xmin":p5, "xmax":p95,"mean":mean}).copyProperties(img, ["system:time_start",'system:id'])

In [8]:
year = 2020
start_year= str(year)
end_year=str(year+1)

collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") \
.filterDate(start_year,end_year)\
.filterBounds(shp_ee) \
.map(maskL457sr)

col1 = collection.map(ndvi)
xmax = col1.select("NDVI").max()
xmin = col1.select("NDVI").min()
mean = col1.select("NDVI").mean()


In [9]:
#研究范围广信区分成乡镇
features = shp_ee.toList(shp_ee.size().getInfo())

In [10]:
for i in range(shp_ee.size().getInfo()):
    feature = ee.Feature(features.get(i))
    roi = feature.geometry()       
    
    Max = properties(xmax).get("xmax").getInfo()
    Min = properties(xmin).get("xmin").getInfo()
    Mean = properties(mean).get("mean").getInfo()
    subtract = Max-Min
    code = feature.get("code").getInfo()

    series = pd.Series({"max":Max,"min":Min,"subtract":subtract,"mean":Mean},name=code)
    df = df.append(series)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [ ]:
# feature = ee.Feature(features.get(1))
# roi = feature.geometry()          
# Max = properties(xmax).get("xmax").getInfo()
# Min = properties(xmin).get("xmin").getInfo()
# Mean = properties(mean).get("mean").getInfo()
# subtract = Max-Min
# code = feature.get("code").getInfo()

# series = pd.Series({"max":Max,"min":Min,"subtract":subtract,"mean":Mean},name=code)
# df = df.append(series)
# print(df)

In [ ]:
# Max.getInfo()

In [11]:
save_pth = r"D:\work\stld\ndvi\1\2020.xlsx"
df.to_excel(save_pth)

In [ ]:
# type(df)

In [ ]:
# df.index

In [ ]:
# df.columns